In [57]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.DEBUG)

In [58]:
from gensim import corpora, models, similarities

In [59]:
import sqlite3
import os
sqlitedb = os.path.join(os.path.expanduser('~'),'Box Sync', 'GradSchoolStuff', 'MastersProject', 'ctpa.sqlite')
if not (os.path.exists(sqlitedb)):
    print("Specified database does not exist")
    sys.exit()

connection = sqlite3.connect(sqlitedb)
with connection:
    cur = connection.cursor()
    cur.execute('select * from reports limit 1')
    col_names = [cn[0] for cn in cur.description]
    rows = cur.fetchall()
    #print(len(rows[0]))
    #print("%s %s %s %s %s %s" % (col_names[0], col_names[1], col_names[2], col_names[3], col_names[4], col_names[5]))

    document = [row[4] for row in rows][0]
    document = document.strip() # remove trailing and leading spaces

In [62]:
import nltk.data

# Load the punkt tokenizer pre-trained on english text
# to improve sentence splitting, would need to create custom tokenizer that understands sections
# however, I think this may be good enoug for now.
sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
output = sent_tokenizer.tokenize(document)
output = [o.strip() for o in output]

# alternatively could use NLTK sentence splitter, but don't like how "we'll" becomes two words "we" and "'ll"
output = [o.lower().split() for o in output]

'''
print(document)
for o in output:
    print('START: ', o)
'''


"\nprint(document)\nfor o in output:\n    print('START: ', o)\n"

In [63]:
from gensim.models import word2vec
# Set values for various parameters, starting point provided by 
# https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-2-word-vectors
num_features = 300    # Word vector dimensionality                      
min_word_count = 5    # Minimum word count, default is 5
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
# default sample = 1e-3   # Downsample setting for frequent words

model = word2vec.Word2Vec(output, \
                          workers=num_workers, \
                          size=num_features, \
                          min_count = min_word_count, \
                          window = context)